In [1]:
import requests
import bs4

In [2]:
import pandas as pd

In [3]:
s = requests.session()

In [4]:
s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.36 Safari/537.36'

In [5]:
s.headers

{'Accept-Encoding': 'gzip, deflate', 'Connection': 'keep-alive', 'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.36 Safari/537.36'}

In [6]:
r = s.get('http://www.row2k.com/results')

In [8]:
soup = bs4.BeautifulSoup(r.content, 'lxml')

In [9]:
# find all individual head to head races with RIT or Rochester Institute of Technology Mentioned
links = list()
for link in soup.findAll('a'):
    if link.has_attr('href'):
        href = link['href']
        if href.startswith('/results/resultspage'):
            t = link.text.lower()
            if 'rochester institute of technology' in t or ' rit' in t:
                links.append(link)

In [10]:
links

[<a href="/results/resultspage.cfm?UID=AF80AF5A922CFE1E6A0A9F3E7343373C&amp;cat=1">Union College vs. RIT</a>,
 <a href="/results/resultspage.cfm?UID=54138AE6C623119EE10DDB345E6EACAC&amp;cat=1">Union College vs. RIT</a>,
 <a href="/results/resultspage.cfm?UID=DE44C7550AC5B96212FED2D3D91E2E53&amp;cat=1">RIT Men vs. SLU Men: Rochester Institute of Technology vs. St. Lawrence University</a>,
 <a href="/results/resultspage.cfm?UID=13043A65B43BDB74D362588F1DC1C38E&amp;cat=1">RIT vs. SLU: Rochester Institute of Technology vs. St. Lawrence University</a>,
 <a href="/results/resultspage.cfm?UID=53CB18C1B62B6DD3F3621C3B7EE30294&amp;cat=1">Hamilton College vs. RIT</a>,
 <a href="/results/resultspage.cfm?UID=F6E4BC1798EF6CBE6AA3FB3CBB65C88E&amp;cat=1">William Smith vs. Hamilton, RIT</a>,
 <a href="/results/resultspage.cfm?UID=1678166CE6DE7E10FEA61DAB9AD723B2&amp;cat=1">Ithaca College vs. U. of Rochester, RIT</a>,
 <a href="/results/resultspage.cfm?UID=B40986A825F09497D1BBEC3AF661E677&amp;cat=1">It

In [11]:
def get_race_results_page(link):
    #TODO: grab date and title
    link.get('href')
    the_soup = bs4.BeautifulSoup(s.get('http://www.row2k.com' + link.get('href')).content, 'lxml')
    results = the_soup.findAll('table', attrs={'border':'0', 'cellpadding':"0", 'cellspacing':"0", 'width':"650"})
    results = results[1].findAll('tr')[4].findAll('tr')[4:-2]
    events = list()
    n = -1
    for r in results:
        if r.has_attr('align'):
            event = {'event_name': r.text, 'participants': list()}
            events.append(event)
            n += 1
        else:
            tds = r.findAll('td')
            if tds[0].text.strip() != '' and len(tds) == 2:
                participant = {'name': tds[0].text.strip(), 'time': tds[1].text.strip()}
                events[n]['participants'].append(participant)
    return {'submitter': the_soup.find('blockquote').i.text, 
            'description': the_soup.find('blockquote').p.text.strip(),
            'results': events}

In [12]:
races = list()
for link in links:
    a = get_race_results_page(link)
    a['Title'] = link.text
    races.append(a)

In [13]:
races

[{'Title': 'Union College vs. RIT',
  'description': "Distance: 1900m Conditions: Light head current, approximately 3 mph cross wind for the Varsity Men's 8 event.\r\nLight head current, approximately 12-14 mph tail wind for the 2V/Novice Men's 8.",
  'results': [{'event_name': '\nVarsity 8:\n',
    'participants': [{'name': 'RIT', 'time': '6:01.41'},
     {'name': 'Union', 'time': '6:16.74'}]},
   {'event_name': '\n2nd Varsity 8:\n',
    'participants': [{'name': 'RIT 2V', 'time': '5:54.31'},
     {'name': 'RIT Novice', 'time': '6:05.91'},
     {'name': 'Union 2V', 'time': '6:26.02'}]}],
  'submitter': 'Submitted by Mike DeWolfe.'},
 {'Title': 'Union College vs. RIT',
  'description': "Distance: 1900m Conditions: Light head current and approximately 3 mph cross wind for Varsity Women event. Light head current approximately 10-12 mph tail wind for 2V/Novice race.  Comments: The Women's 2V/Novice event was started 2 lengths into course on float.",
  'results': [{'event_name': '\nVarsity